In [1]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text
from traitlets import link, dlink

In [2]:
import ase
import ase.io
from itertools import chain
import numpy

In [3]:
a=ase.io.read('PAD_1-512.traj')

In [4]:
a

Atoms(symbols='Fe13Pt42', pbc=False, cell=[20.0, 20.0, 20.0])

In [5]:
a.get_cell(), a.get_positions()[0]

(array([[ 20.,   0.,   0.],
        [  0.,  20.,   0.],
        [  0.,   0.,  20.]]), array([ 10.,  10.,  10.]))

In [6]:
cm=a.get_center_of_mass()

verts=list(chain.from_iterable([[[0,0,0],list(v)] for v in numpy.diag([1,1,1])]))
coordgeom = PlainGeometry(vertices=verts,
                          colors = 2*['red'] + 2*['green'] + 2*['blue'])
coord = Line(geometry=coordgeom, 
             material=LineBasicMaterial(linewidth=2, vertexColors='VertexColors'), 
             type='LinePieces')

verts=[]
for sp in [[0,0,0],[1,1,0],[1,0,1],[0,1,1]]:
    v0=numpy.dot(a.get_cell(),numpy.array(sp))
    c=-2*numpy.array(sp)+1
    verts+=list(chain.from_iterable([[list(-cm+v0),list(s*v-cm+v0)] for s,v in zip(c,a.get_cell())]))

    
ucgeom = PlainGeometry(vertices=verts, colors = len(verts)*['black'])
uc = Line(geometry=ucgeom, 
             material=LineBasicMaterial(linewidth=1, vertexColors='VertexColors'), 
             type='LinePieces')

atoms = [ Mesh(geometry=SphereGeometry(radius=1), 
                                    material=LambertMaterial(color='red'),
                                    position=list(p-cm)) for p in a.get_positions()]



scene = Scene(children=atoms + [uc, DirectionalLight(color='#ccaabb', position=[0,40,40]),AmbientLight(color='#cccccc')])
c = PerspectiveCamera(position=[0, 30, 30])
renderer = Renderer(camera=c, scene = scene, controls=[OrbitControls(controlling=c)])
display(renderer)

In [7]:
t=ase.io.read('PAD_1-512.traj',index=':')

In [9]:
import time
for a1 in t:
    for ao, ap in zip(atoms, a1.get_positions()):
        ao.position=list(ap-cm)
        time.sleep(.05)

KeyboardInterrupt: 